Для входа на сайт будем использовать Selenium

In [1]:
! pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
from fake_useragent import UserAgent

In [3]:
options = webdriver.ChromeOptions()

In [4]:
useragent = UserAgent()

In [5]:
options.add_argument(f"user-agent={useragent.random}")

In [6]:
# Заходим через прокси, т.к. сайт через РФ не работает
options.add_argument("--proxy-server=45.153.20.233:10324")

In [7]:
driver = Chrome(executable_path='D:\chromedriver', options=options)

C:\Users\М\AppData\Local\Temp\ipykernel_15604\376164925.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(executable_path='D:\chromedriver', options=options)


In [8]:
#
driver.get("https:\\ru.linkedin.com")

Далее вручную вводим логин и пароль

Вводим название вакансии для поиска

In [10]:
search_vacancy_name = input()

Data Scientist


In [ ]:
search_region_name = input()

In [ ]:
# Вводим название решиона в поисковой строке
driver.find_element("xpath", '//*[@id="jobs-search-box-location-id-ember164"]').send_keys(search_region_name)

In [ ]:
# Ищем вакансию в поисковой строке
driver.find_element("xpath", '//*[@id="jobs-search-box-keyword-id-ember4286"]').send_keys(search_vacancy_name, Keys.ENTER)

In [ ]:
# Добавим несколько для уточнения посика
driver.find_element("xpath", '//*[@id="search-reusables__filters-bar"]/div/div/button').click()
driver.find_element("xpath", '//*[@id="ember4796"]/ul/li[12]/fieldset/div/ul/li[1]/label').click()
driver.find_element("xpath", '//*[@id="ember4796"]/ul/li[12]/fieldset/div/ul/li[5]/label').click()

In [ ]:
driver.find_element("xpath", '//*[@id="ember4845"]/span').click()

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep

In [ ]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
from selenium.webdriver.common.by import By

Для начала отработает парсинг для одной вакансии, чтобы понимать, откуда и какие данные подтягивать

In [ ]:
driver.find_element(By.CLASS_NAME, 'job-card-container').click()

In [ ]:
vacancy_name = soup.find('h2', class_='t-24 t-bold jobs-unified-top-card__job-title').text.strip()
vacancy_name

In [ ]:
company_name = soup.find('span', class_='jobs-unified-top-card__company-name').text.strip()
company_name

In [ ]:
city = soup.find('span', class_='jobs-unified-top-card__bullet').text.strip()
city

In [ ]:
job_type = []

job_types = soup.find('li', class_='jobs-unified-top-card__job-insight').find_all('span')

for t in job_types:
    job_type.append(t.text)
    
#job_type[0][1:-1]
job_type

In [ ]:
date = soup.find('span', class_='jobs-unified-top-card__posted-date').text.strip()

date

In [ ]:
description = soup.find('div', class_='jobs-description__content jobs-description-content').text.strip()
description

Создадим цикл, который прогонит по всем страницам и вакансиям

In [ ]:
data=[]

In [ ]:
for p in range(0,975,25):
    sleep(3)
    driver.get(f"https://www.linkedin.com/jobs/search/?currentJobId=3128810971&f_T=340&geoId=103644278&keywords=Data%20Scientist&location=%D0%A1%D0%BE%D0%B5%D0%B4%D0%B8%D0%BD%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%A8%D1%82%D0%B0%D1%82%D1%8B%20%D0%90%D0%BC%D0%B5%D1%80%D0%B8%D0%BA%D0%B8&refresh=true&sortBy=R&start={p}")

    for i in range(0,25):
        
        driver.find_element(By.CLASS_NAME, f'jobs-search-two-pane__job-card-container--viewport-tracking-{i}').click()
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")

        soup = BeautifulSoup(driver.page_source, 'lxml')

        vacancy_name = soup.find('h2', class_='jobs-unified-top-card__job-title').text.strip()
        company_name = soup.find('span', class_='jobs-unified-top-card__company-name').text.strip()
        city = soup.find('span', class_='jobs-unified-top-card__bullet').text.strip()
        date = soup.find('span', class_='jobs-unified-top-card__posted-date').text.strip()
        try:
            workplace_type = soup.find('span', class_='jobs-unified-top-card__workplace-type').text.strip()
        except:
            workplace_type = ''

        job_type = []

        try:
            job_types = soup.find('li', class_='jobs-unified-top-card__job-insight').find('span')
            for t in job_types:
                job_type.append(t.text)
        except:
            job_type = []

        description = soup.find('div', class_='jobs-description__content jobs-description-content').text.strip()


        data.append([vacancy_name, company_name, city, date, job_type, description, workplace_type])
    
    print(f'датасет состоит из {len(data)} строк')
        
print(f'итоговый датасет состоит из {len(data)} строк')

In [ ]:
header  = ['vacancy_name', 'company_name', 'city', 'date', 'job_type', 'description', 'workplace_type']

In [ ]:
df = pd.DataFrame(data, columns=header)
len(df)

In [ ]:
df

In [ ]:
# Сохраняем датасет в формат csv
df.to_csv('D:\Учеба\Магистратура\Дататон\linkedin_ds_1.csv')